# 株価の時系列データの分析

pandas.DataFrame.rolling　https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html  <br>
mplfinance   https://github.com/matplotlib/mplfinance

#### 2020年9月よりローソク足チャートのパッケージとAPIを変更
従来はmatplotlibが提供していたが廃止となった。この変更の詳細は下記のローソク足チャートの箇所に記述した。

In [ ]:
# -*- coding: utf-8 -*-
# Rolling mean statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

#import matplotlib.finance as mpf
#from mpl_finance import candlestick_ohlc
from matplotlib.dates import date2num

import mplfinance as mpf
%matplotlib inline

matplotlib.rcParams['font.family'] ='Yu Mincho' # matplotlibで日本語を使う
FLAG_fig = False

In [ ]:
#url = 'https://sites.google.com/site/datasciencehiro/datasets/Stock_7203.txt'
url = 'datasets/Stock_7203.txt'
df = pd.read_csv(url, index_col='日付', parse_dates=[0] )
df = df.sort_index() # 日付を昇順に並び替える
df.head()

In [ ]:
item='終値'
W = 25
ax = df[item].plot(color='black', label=item)

rol = df[item].rolling(window=W).mean()
rol.plot(ax=ax, label='移動平均25日', color='red', linestyle='dashed')

ewm = df[item].ewm(span=W).mean()
ewm.plot(ax=ax, label='指数移動平均スパン25', color='green', linestyle='dashed')

plt.grid()
plt.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_result_1.png')
plt.show()

#### 移動平均,　windowの大きさの影響
W＝7日と28日

In [ ]:
item='終値'
ax = df[item].plot(color='black', label=item)

rol1 = df[item].rolling(window=7).mean()
rol1.plot(ax=ax, label='移動平均7日', color='red', linestyle='dashed')

rol2 = df[item].rolling(window=28).mean()
rol2.plot(ax=ax, label='移動平均28日', color='green', linestyle='dashed')

plt.grid()
plt.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_result_2.png')
plt.show()

#### ボリンジャーバンド（Bollinger bands)

In [ ]:
item = '終値'
W = 25
ax = df[item].plot(label=item, color='black', linestyle='solid')
rolm = df[item].rolling(window=W).mean()
rolst = df[item].rolling(window=W).std() #unbiased
m = 2
upper_band = rolm + rolst*m
lower_band = rolm - rolst*m

rolm.plot(ax=ax, label='移動平均25日', color='green', linestyle='solid')
upper_band.plot(ax=ax, label='Upper band', color='red', linestyle='dashed')
lower_band.plot(ax=ax, label='Lower band', color='blue', linestyle='dashed')

plt.grid()
plt.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_BollingerBand.png')
plt.show()

#### ローソク足チャート
・もともと，matplotlib.financeにあったが，現在はmaplotlibは提供していない。<br>
・2020年9月時点，右のパッケージを使用：　https://github.com/matplotlib/mplfinance<br>
・使い方は，このサイトを参照されたい。
・dataframeのラベル名が英語で参照のため，必要なデータだけに対しての英語名への変換を次で行う。

In [ ]:
dfE = df.drop(columns='売買代金', axis=1)
dfE.rename(columns={'始値':'Open','高値':'High','安値':'Low','終値':'Close','出来高':'Volume'}, inplace=True)
dfE = dfE.rename({'日付':'Date'})
dfE.index.name='Date'
dfE.head(3)

#### 使用しているmplfinanceは，ローソク足チャートと出来高（volume）を同時に見られるようになった。

In [ ]:
dfESep = dfE['20170901':'20170930'] # 9月の1か月だけを見る
mpf.plot(dfESep, type='candle', volume=True)

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_Plot_CandleStick_1.png')

### 備考　nullデータの対処
・Yafooファナンスから，日経データをダウンロード。
・ファイル名"^N225.csv"，エンコードはutf-8である。
・データにはnullデータ（マーケットが閉じている日は取引が無い）がある<br>
・pandasはnull（データは無いという意味）をNan(Not a Number, 数字データは無いという意味)と解釈する。
・この対処法を示す
・原データを格納する変数（データフレーム）をdf0とする<br>


In [ ]:
#url = 'https://sites.google.com/site/datasciencehiro/datasets/^N225.csv'
url = 'datasets/^N225.csv'
df0 = pd.read_csv(url, index_col='Date', parse_dates=[0] )
df0 = df0.sort_index() # 日付を昇順に並び替える
df0.head()

#### 原データからNan(nullデータ)を削除して，これを変数dfに代入する
メッソド".notnull()"はNan(null)データが無い行を返す。  
データが無い日は，Open, High, Low, Close　全て同じであるから，ここでは'Open'を用いる 

In [ ]:
df = df0[df0['Open'].notnull()]
df.head()

#### 移動平均

In [ ]:
item='Close'
ax = df[item].plot(color='black', label=item)

rol1 = df[item].rolling(window=5).mean()
rol1.plot(ax=ax, label='Window=5', color='red', linestyle='dashed')

rol2 = df[item].rolling(window=25).mean()
rol2.plot(ax=ax, label='Window=25', color='green', linestyle='dashed')


plt.grid()
plt.legend(loc='upper right')
plt.show()